In [ ]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import matplotlib.font_manager as fm
from sklearn.datasets import load_boston

# # GPU 설정 : 런타임 > 런타임 유형 변경 > GPU
# # GPU 사용 가능한지 확인
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
# 데이터 프레임화
bst = load_boston()

bst_x=pd.DataFrame(data=bst.data,columns=bst.feature_names)
bst_y=pd.DataFrame(bst.target,columns=["Meds"])
bst=pd.merge(bst_x,bst_y,left_index=True,right_index=True)
bst

In [ ]:
# OLS 모델
model_bst=sm.OLS(bst_y,bst_x)
result_bst=model_bst.fit()

In [ ]:
sns.regplot(x=bst["RM"],y=bst["Meds"])
# sns.regplot(x=bst["INDUS"],y=bst["Meds"])
plt.show()

In [ ]:
'''
endog: 종속변수 문자열

exog_i: 분석 대상이 되는 독립변수 문자열

exog_others: 나머지 독립변수 문자열의 리스트

data: 모든 데이터가 있는 데이터프레임

obs_labels: 데이터 라벨링 여부

ret_coords: 잔차 데이터 반환 여부

부분회귀 플롯으로 살펴보면 AGE 변수와 종속변수는 상관관계가 없다는 것을 알 수 있다.
부분회귀 플롯에서 가로축의 값은 독립변수 자체의 값이 아니라 어떤 독립변수에서 다른 독립변수의 영향을 제거한 일종의 “순수한 독립변수 성분”을 뜻한다.
sm.graphics.plot_partregress_grid 명령을 쓰면 전체 데이터에 대해 한번에 부분회귀 플롯을 그릴 수 있다.
'''

others=list(set(bst.columns).difference(set(["Meds","RM"])))
others
p,resid=sm.graphics.plot_partregress('Meds',
                                     "RM",
                                     others,
                                     data=bst,
                                     obs_labels=False,
                                     ret_coords=True)
plt.show()


In [ ]:
'''
특정한 독립변수 x2를 제외한 나머지 독립변수 X1들로 종속변수 y를 선형 회귀분석하여 잔차 y∗를 구한다.

특정한 독립변수 x2를 제외한 나머지 독립변수 X1들로 특정한 독립변수 x2를 선형 회귀분석하여 잔차 x∗2를 구한다.

잔차 x∗2를 독립변수로, 잔차 y∗를 종속변수로 하여 선형 회귀분석한다.
'''
fig = plt.figure(figsize=(8, 20))
sm.graphics.plot_partregress_grid(result_bst, fig=fig)

plt.show()

## CCPR 플롯
CCPR(Component-Component plus Residual) 플롯도 부분회귀 플롯과 마찬가지로 특정한 하나의 변수의 영향을 살펴보기 위한 것이다.

다음과 같은 회귀 모형이 있다고 가정하자.

y=y^+e=w1x1+w2x2+⋯+wixi+⋯+wKxK+e
CCPR 플롯은 이 성분 중에서

xi를 가로축으로

wixi+e을 세로축으로

그린 스캐터 플롯이다.

In [ ]:
# CCPR 플롯
# result: 회귀분석 결과 객체
# exog_idx: 분석 대상이 되는 독립변수 문자열

sm.graphics.plot_ccpr(result_bst, "AGE")
plt.show()
'''
CCPR 플롯에서는 부분회귀 플롯과 달리 독립변수가 원래의 값 그대로 나타난다.

마찬가지로 sm.graphics.plot_ccpr_grid 명령을 쓰면 전체 데이터에 대해 한번에 CCPR 플롯을 그릴 수 있다.

plot_ccpr_grid 명령은 모든 독립변수에 대해 CCPR 플롯을 그려준다.
'''

In [ ]:
fig = plt.figure(figsize=(8, 15))
sm.graphics.plot_ccpr_grid(result_bst, fig=fig)
plt.show()

In [ ]:
# plot_regress_exog(result, exog_idx) 명령은 부분회귀 플롯과 CCPR을 같이 보여준다.
# result: 회귀분석 결과 객체
# exog_idx: 분석 대상이 되는 독립변수 문자열
fig = sm.graphics.plot_regress_exog(result_bst, "AGE")
plt.tight_layout(pad=4, h_pad=0.5, w_pad=0.5)
plt.show()